In [1]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import normalize
from scipy.sparse import dia_matrix
from scipy.optimize import minimize, shgo, dual_annealing, differential_evolution, basinhopping
import scipy

from sklearn.model_selection import train_test_split

In [2]:
path = "../data/data.xlsx"
df = pd.read_excel(path)
df.Дата = df.Дата.astype('datetime64[ns]')

In [3]:
features = [
    ['intercept', 'Кривизна'],
    ['intercept', 'Кривизна', 'Профиль пути'],
    ['intercept', 'Кривизна', 'n_feature_1'], # Профиль пути * Макс. число вагонов в сходе
    ['intercept', 'Кривизна', 'n_feature_2'], # 1 - Макс. число вагонов в сходе / Общее количество вагонов
    ['intercept', 'Кривизна', 'n_feature_3'], # Скорость * Загрузка
    ['intercept', 'Кривизна', 'Профиль пути', 'n_feature_3'],
    ['intercept', 'Кривизна', 'Профиль пути', 'n_feature_3', 'n_feature_2'],
    ['intercept', 'Профиль пути', 'n_feature_3', 'n_feature_2']
]

target = ['Количество сшедших вагонов']

In [58]:
class MLM():
    
    neg_log_likelihood_fun = 0
    const_log_likelihood_fun = 0
    
    predict_fun = None
    
    optimization_method = shgo
    borders = 10 * [(-1000, 1000)]
    
    df = None
    features = None
    target = None
    
    count_of_param_fun = 0
    
    def __init__(self, df, features, target,
                 neg_log_likelihood_fun, const_log_likelihood_fun,
                optimization_method, borders, predict_fun,
                count_of_param_fun):
        self.neg_log_likelihood_fun = neg_log_likelihood_fun
        self.const_log_likelihood_fun = const_log_likelihood_fun
        self.optimization_method = optimization_method
        self.borders = borders
        self.predict_fun = predict_fun
        self.count_of_param_fun = count_of_param_fun
        self.features = features
        self.target = target
        self.df = df
    
    def const_log_likehood_zero(y):
        return 0
    
    def calc_aic_c(self, logL, feature) -> float:
        logL = float(logL[2:-2])
        k = len(feature)
        n = len(self.y)
        return round(2 * (k - logL) + 2 * k * (k + 1) / (n - k - 1), 2)
    
    def to_fixed2(self, a, is_tex_output=False):
        result_str = ''
        for i in a:
            if is_tex_output:
                result_str += ('$%.2f$, ' % i)
            else:
                result_str += ('%.2f, ' % i)
        result_str = '[' + result_str[:-2] + ']'
        return result_str
    
    def fit_all(self, get_tex_code=False):
        num_of_feature = 0
        for feature in self.features:
            num_of_feature += 1

            data = self.df[feature + self.target].dropna()
            self.y = data[self.target]
            self.y = np.array(self.y)
            self.X = data.drop(self.target, axis=1)
            self.X = np.matrix(self.X)
            
            thetas_for_tex = ''
            thetas_without_round_for_tex = ''
            AIC_c_for_tex = ''
            
            print('Признаковое пространство', num_of_feature, ':', feature)
            print('Мощность выборки n = ', len(self.y))
            
            for param_index in range(self.count_of_param_fun):
                
                count_of_attempt = 0
                result = self.optimization_method(self.neg_log_likelihood_fun,
                        self.borders[:len(feature)],
                        args=(self.X, self.y, param_index))
                
                while math.isnan(result['fun']) and count_of_attempt < 15:
                    result = self.optimization_method(self.neg_log_likelihood_fun,
                            self.borders[:len(feature)],
                            args=(self.X, self.y, param_index))
                    count_of_attempt += 1
                
                if get_tex_code:
                    thetas_for_tex += self.to_fixed2(result['x'], is_tex_output=True) + ' & '
                    thetas_without_round_for_tex += str(result['x']) + ' & '
                    AIC_c_for_tex += '$' + str(self.calc_aic_c(self.to_fixed2([-result['fun'] + self.const_log_likelihood_fun(self.y)], is_tex_output=True), feature)) + '$' + ' & '
                
                print('success: ', result['success'])
                print('ln L: ', self.to_fixed2([-result['fun'] + self.const_log_likelihood_fun(self.y)]))
                print('AIC_c', self.calc_aic_c(self.to_fixed2([-result['fun'] + self.const_log_likelihood_fun(self.y)]), feature))
                print('thetas: ', self.to_fixed2(result['x']))
                print()
            
            if get_tex_code:
                print('latex code:')
                print('thetas_for_tex:', thetas_for_tex[:-2], '\n')
                print('thetas_for_tex:', thetas_without_round_for_tex[:-2], '\n')
                print('AIC_c_for_tex: ', AIC_c_for_tex[:-2], '\n')
            print('########################################################')
            
    
    # TO DO
    def predict(self, x_pred):
        return 0

In [4]:
lambdas = [
    (lambda t: np.exp(t)),
    (lambda t: np.exp(-t * t)),
    (lambda t: 1 + np.sqrt(np.abs(25 - (t-5)*(t-5)))),
    (lambda t: (t-1)*(t-1)),
    (lambda t: 1 / (1 + t*t)),
    (lambda t: t * (np.pi / 2.0 - np.arctan(-t)) + 1),
    (lambda t: np.log(1 + t*t) + 1),
]

log_lambdas = [
    (lambda t: t),
    (lambda t: -t * t),
    (lambda t: np.log(1 + np.sqrt(np.abs(25 - (t-5)*(t-5))))),
    (lambda t: np.log((t-1)*(t-1))),
    (lambda t: -np.log(1 + t*t)),
    (lambda t: np.log(t * (np.pi / 2.0 - np.arctan(-t)) + 1)),
    (lambda t: np.log(np.log(1 + t*t) + 1)),
]

def poisson_neg_log_likelihood_function(theta, X, y, lambda_index):
    result = 0
    for i in range(len(y)):
        dot_prod = float(np.dot(X[i], theta))
        lambda_value = lambdas[lambda_index](dot_prod)
        log_lambda_value = log_lambdas[lambda_index](dot_prod)
        result += (float(y[i]) - 1) * log_lambda_value - lambda_value
    return -result

def poisson_get_log_likelihood_const(y):
        result = 0
        for i in range(len(y)):
            result += -np.log(scipy.special.factorial(y[i]))
        return result

In [59]:
poisson_regressor = MLM(df, features, target,
                        poisson_neg_log_likelihood_function,
                        poisson_get_log_likelihood_const,
                        shgo, 10*[(-1000,1000)], None, len(lambdas))
poisson_regressor.fit_all(get_tex_code=True)

Признаковое пространство 1 : ['intercept', 'Кривизна']
Мощность выборки n =  46
success:  True
ln L:  [-256.27]
AIC_c -508.12
thetas:  [1.18, -271.63]

success:  True
ln L:  [-306.66]
AIC_c -608.92
thetas:  [-0.00, 0.00]

success:  True
ln L:  [-306.66]
AIC_c -608.92
thetas:  [0.00, 0.00]

success:  True
ln L:  [-256.14]
AIC_c -507.92
thetas:  [-0.80, 208.15]

success:  True
ln L:  [-306.66]
AIC_c -608.92
thetas:  [-0.00, 0.00]



<ipython-input-4-3655a5e8d18a>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),


success:  True
ln L:  [-256.28]
AIC_c -508.12
thetas:  [0.95, -236.29]

success:  True
ln L:  [-256.66]
AIC_c -508.92
thetas:  [2.66, -549.73]

latex code:
thetas_for_tex: [$1.18$, $-271.63$] & [$-0.00$, $0.00$] & [$0.00$, $0.00$] & [$-0.80$, $208.15$] & [$-0.00$, $0.00$] & [$0.95$, $-236.29$] & [$2.66$, $-549.73$]  

thetas_for_tex: [   1.17663977 -271.62551142] & [-2.73694797e-11  0.00000000e+00] & [0. 0.] & [ -0.79854568 208.15177928] & [-2.73694797e-11  0.00000000e+00] & [   0.94835424 -236.2893394 ] & [   2.66181393 -549.7289914 ]  

AIC_c_for_tex:  $516.82$ & $617.6$ & $617.6$ & $516.56$ & $617.6$ & $516.84$ & $517.6$  

########################################################
Признаковое пространство 2 : ['intercept', 'Кривизна', 'Профиль пути']
Мощность выборки n =  41


<ipython-input-4-3655a5e8d18a>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),


success:  True
ln L:  [-242.56]
AIC_c -478.35
thetas:  [1.17, -160.85, 39.30]

success:  True
ln L:  [-299.87]
AIC_c -592.95
thetas:  [-0.00, 0.00, 0.00]

success:  True
ln L:  [-239.22]
AIC_c -471.75
thetas:  [0.56, -114.67, 35.95]

success:  True
ln L:  [-239.73]
AIC_c -472.75
thetas:  [-0.83, 212.70, -56.93]

success:  True
ln L:  [-299.87]
AIC_c -592.95
thetas:  [-0.00, 0.00, 0.00]

success:  True
ln L:  [-238.18]
AIC_c -469.55
thetas:  [1.05, -281.75, 83.44]

success:  True
ln L:  [-235.22]
AIC_c -463.75
thetas:  [3.28, -486.77, 276.36]

latex code:
thetas_for_tex: [$1.17$, $-160.85$, $39.30$] & [$-0.00$, $0.00$, $0.00$] & [$0.56$, $-114.67$, $35.95$] & [$-0.83$, $212.70$, $-56.93$] & [$-0.00$, $0.00$, $0.00$] & [$1.05$, $-281.75$, $83.44$] & [$3.28$, $-486.77$, $276.36$]  

thetas_for_tex: [   1.17290277 -160.84988797   39.29626948] & [-2.73694797e-11  0.00000000e+00  0.00000000e+00] & [   0.55589183 -114.66800927   35.94991553] & [ -0.83371448 212.69978342 -56.92629995] & [-2.73

...

success:  True
ln L:  [-247.44]
AIC_c -485.55
thetas:  [9.24, 3.94, 3.55, -0.11]

success:  True
ln L:  [-209.23]
AIC_c -409.15
thetas:  [0.29, 244.79, -52.47, -0.02]

success:  True
ln L:  [-283.07]
AIC_c -556.75
thetas:  [-0.00, 0.00, 0.00, -0.00]

success:  True
ln L:  [-208.26]
AIC_c -407.15
thetas:  [-0.47, -328.73, 75.88, 0.03]

success:  True
ln L:  [-213.65]
AIC_c -417.95
thetas:  [-0.31, -633.78, 7.19, 0.08]

latex code:
thetas_for_tex: [$-0.15$, $-148.04$, $53.13$, $0.03$] & [$-0.00$, $0.00$, $0.00$, $-0.00$] & [$9.24$, $3.94$, $3.55$, $-0.11$] & [$0.29$, $244.79$, $-52.47$, $-0.02$] & [$-0.00$, $0.00$, $0.00$, $-0.00$] & [$-0.47$, $-328.73$, $75.88$, $0.03$] & [$-0.31$, $-633.78$, $7.19$, $0.08$]  

thetas_for_tex: [-1.48804956e-01 -1.48036460e+02  5.31272408e+01  2.66695834e-02] & [-7.09391652e-14  0.00000000e+00  0.00000000e+00 -2.01254412e-10] & [ 9.24393014  3.94187122  3.55281838 -0.11333238] & [ 2.93385652e-01  2.44792516e+02 -5.24674773e+01 -2.44099749e-02] & [-7.0709

<ipython-input-4-3655a5e8d18a>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),


ln L:  [-281.07]
AIC_c -549.93
thetas:  [-0.00, 0.00, 0.00, -0.00, 0.00]

success:  True
ln L:  [-281.07]
AIC_c -549.93
thetas:  [0.00, 0.00, 0.00, 0.00, 0.00]

success:  True
ln L:  [-194.73]
AIC_c -377.33
thetas:  [-0.98, 242.26, -22.62, -0.01, 1.73]

success:  True
ln L:  [-281.07]
AIC_c -549.93
thetas:  [-0.00, 0.00, 0.00, -0.00, 0.00]

success:  True
ln L:  [-194.63]
AIC_c -377.13
thetas:  [1.28, -373.54, 41.10, 0.02, -2.34]

success:  True
ln L:  [-198.76]
AIC_c -385.33
thetas:  [2.69, -57.95, 235.44, 0.08, -8.15]

latex code:
thetas_for_tex: [$0.92$, $-105.35$, $34.34$, $0.02$, $-2.24$] & [$-0.00$, $0.00$, $0.00$, $-0.00$, $0.00$] & [$0.00$, $0.00$, $0.00$, $0.00$, $0.00$] & [$-0.98$, $242.26$, $-22.62$, $-0.01$, $1.73$] & [$-0.00$, $0.00$, $0.00$, $-0.00$, $0.00$] & [$1.28$, $-373.54$, $41.10$, $0.02$, $-2.34$] & [$2.69$, $-57.95$, $235.44$, $0.08$, $-8.15$]  

thetas_for_tex: [ 9.20623915e-01 -1.05351925e+02  3.43369654e+01  2.12149958e-02
 -2.23822805e+00] & [-6.99064063e-14 

<ipython-input-4-3655a5e8d18a>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),


AIC_c -400.95
thetas:  [-0.49, -15.96, -0.02, 1.44]

success:  True
ln L:  [-283.07]
AIC_c -556.75
thetas:  [-0.00, 0.00, -0.00, 0.00]

success:  True
ln L:  [-205.19]
AIC_c -400.95
thetas:  [0.31, 26.23, 0.03, -1.97]

success:  True
ln L:  [-204.94]
AIC_c -400.55
thetas:  [1.91, 281.49, 0.08, -6.90]

latex code:
thetas_for_tex: [$0.55$, $39.28$, $0.03$, $-2.14$] & [$-0.00$, $0.00$, $-0.00$, $0.00$] & [$0.00$, $0.00$, $0.00$, $0.00$] & [$-0.49$, $-15.96$, $-0.02$, $1.44$] & [$-0.00$, $0.00$, $-0.00$, $0.00$] & [$0.31$, $26.23$, $0.03$, $-1.97$] & [$1.91$, $281.49$, $0.08$, $-6.90$]  

thetas_for_tex: [ 5.51509058e-01  3.92803528e+01  2.54177579e-02 -2.13831493e+00] & [-7.07737609e-14  0.00000000e+00 -2.01492897e-10  0.00000000e+00] & [0. 0. 0. 0.] & [ -0.49176887 -15.96251758  -0.01698865   1.43914189] & [-7.04401905e-14  0.00000000e+00 -2.00519742e-10  0.00000000e+00] & [ 0.31226161 26.22997155  0.02698085 -1.96985398] & [ 1.90837587e+00  2.81493009e+02  7.62296768e-02 -6.89972218e+00

In [5]:
lambdas_2 = [
    (lambda t: np.exp(t)),
    (lambda t: np.exp(-t * t)),
    (lambda t: 1 / (1 + np.exp(-t))),
    #(lambda t: (t-1)*(t-1)),
    (lambda t: 1 / (1 + t*t)),
    (lambda t: t * (np.pi / 2.0 - np.arctan(-t)) + 1),
    #(lambda t: np.log(1 + t*t) + 1),
]

log_lambdas_2 = [
    (lambda t: t),
    (lambda t: -t * t),
    (lambda t: -np.log(1 + np.exp(-t))),
    #(lambda t: np.log((t-1)*(t-1))),
    (lambda t: -np.log(1 + t*t)),
    (lambda t: np.log(t * (np.pi / 2.0 - np.arctan(-t)) + 1)),
    #(lambda t: np.log(np.log(1 + t*t) + 1)),
]

log_one_minus_lambdas_2 = [
    (lambda t: np.log(1 - np.exp(t))),
    (lambda t: np.log(1 - np.exp(-t * t))),
    (lambda t: -np.log(1 + np.exp(t))),
    #(lambda t: np.log(1 - (t-1)*(t-1))),
    (lambda t: np.log(t*t) - np.log(1 + t*t)),
    (lambda t: np.log(-t * (np.pi / 2.0 - np.arctan(-t)))),
    #(lambda t: np.log(-np.log(1 + t*t))),
]

def geometry_neg_log_likelihood_function(theta, X, y, lambda_index):
    result = 0
    for i in range(len(y)):
        dot_prod = float(np.dot(X[i], theta))
        lambda_value = lambdas_2[lambda_index](dot_prod)
        log_lambda_value = log_lambdas_2[lambda_index](dot_prod)
        result += (float(y[i]) - 1) * log_one_minus_lambdas_2[lambda_index](dot_prod) + log_lambda_value
    return -result

def geometry_get_log_likelihood_const(y):
        return 0

In [60]:
geometry_regressor = MLM(df, features, target,
                        geometry_neg_log_likelihood_function,
                        geometry_get_log_likelihood_const,
                        dual_annealing, 10*[(-1000,1000)], None, len(lambdas_2))
geometry_regressor.fit_all(get_tex_code=True)

Признаковое пространство 1 : ['intercept', 'Кривизна']
Мощность выборки n =  46


<ipython-input-5-49b39c550010>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),
<ipython-input-5-49b39c550010>:22: RuntimeWarning: overflow encountered in exp
  (lambda t: np.log(1 - np.exp(t))),
<ipython-input-5-49b39c550010>:22: RuntimeWarning: invalid value encountered in log
  (lambda t: np.log(1 - np.exp(t))),


success:  True
ln L:  [-98.07]
AIC_c -191.72
thetas:  [-1.46, 215.57]

success:  True
ln L:  [-98.19]
AIC_c -191.92
thetas:  [1.20, -90.37]



<ipython-input-5-49b39c550010>:24: RuntimeWarning: overflow encountered in exp
  (lambda t: -np.log(1 + np.exp(t))),
<ipython-input-10-1c2d512b1abc>:7: RuntimeWarning: invalid value encountered in double_scalars
  result += (float(y[i]) - 1) * log_one_minus_lambdas_2[lambda_index](dot_prod) + log_lambda_value
<ipython-input-5-49b39c550010>:4: RuntimeWarning: overflow encountered in exp
  (lambda t: 1 / (1 + np.exp(-t))),
<ipython-input-5-49b39c550010>:14: RuntimeWarning: overflow encountered in exp
  (lambda t: -np.log(1 + np.exp(-t))),


success:  True
ln L:  [-98.21]
AIC_c -192.12
thetas:  [-1.18, 278.32]

success:  True
ln L:  [-98.04]
AIC_c -191.72
thetas:  [1.82, -233.20]



...

success:  True
ln L:  [-729.05]
AIC_c -1448.75
thetas:  [-51.83, -1000.00, 0.52, 18.16]

success:  True
ln L:  [-84.25]
AIC_c -159.15
thetas:  [-0.18, -49.54, -0.03, 1.64]



<ipython-input-5-49b39c550010>:24: RuntimeWarning: overflow encountered in exp
  (lambda t: -np.log(1 + np.exp(t))),
<ipython-input-10-1c2d512b1abc>:7: RuntimeWarning: invalid value encountered in double_scalars
  result += (float(y[i]) - 1) * log_one_minus_lambdas_2[lambda_index](dot_prod) + log_lambda_value
<ipython-input-5-49b39c550010>:4: RuntimeWarning: overflow encountered in exp
  (lambda t: 1 / (1 + np.exp(-t))),
<ipython-input-5-49b39c550010>:14: RuntimeWarning: overflow encountered in exp
  (lambda t: -np.log(1 + np.exp(-t))),
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:697: RuntimeWarning: invalid value encountered in double_scalars
  df = (f(*((xk + d,) + args)) - f0) / d[k]


success:  True
ln L:  [-77.49]
AIC_c -145.55
thetas:  [-0.59, -15.12, -0.02, 1.88]

success:  True
ln L:  [-73.01]
AIC_c -136.75
thetas:  [0.34, 99.11, 0.05, -3.08]



<ipython-input-5-49b39c550010>:27: RuntimeWarning: invalid value encountered in log
  (lambda t: np.log(-t * (np.pi / 2.0 - np.arctan(-t)))),


success:  True
ln L:  [-171.83]
AIC_c -334.35
thetas:  [-19.40, -550.73, 0.17, 8.78]

latex code:
thetas_for_tex: [$-51.83$, $-1000.00$, $0.52$, $18.16$] & [$-0.18$, $-49.54$, $-0.03$, $1.64$] & [$-0.59$, $-15.12$, $-0.02$, $1.88$] & [$0.34$, $99.11$, $0.05$, $-3.08$] & [$-19.40$, $-550.73$, $0.17$, $8.78$]  

thetas_for_tex: [-5.18348251e+01 -9.99999013e+02  5.19194003e-01  1.81625485e+01] & [-1.80962034e-01 -4.95376748e+01 -2.74018506e-02  1.63735997e+00] & [ -0.58750862 -15.12032606  -0.0230543    1.882114  ] & [ 3.42843809e-01  9.91125989e+01  4.87375594e-02 -3.08178969e+00] & [-1.94041628e+01 -5.50726479e+02  1.68875182e-01  8.78386225e+00]  

AIC_c_for_tex:  $1467.35$ & $177.75$ & $164.23$ & $155.27$ & $352.91$  

########################################################
